# Modules

In [1]:
%pylab --no-import-all inline
#import EKPSolvers
import gurobipy as grb
import EKPSolvers as ekp

Populating the interactive namespace from numpy and matplotlib


# Initialization

In [2]:
n = 6
np.random.seed(15)
c = 1 + 0.3*np.random.randn(n)
w = np.random.randint(5,12,size=n)+1
x = np.array([1, 0, 1, 1, 0, 0])#np.random.randint(2,size=n)
rhs = sum(w*x)

print(c.tolist(),w.tolist(),x.tolist(),rhs,sep='\n')

vtype = grb.GRB.BINARY

[0.9063014555369366, 1.1017854118138553, 0.953227439835186, 0.8494630980106754, 1.0706706675178057, 0.470918422958737]
[11, 7, 11, 6, 12, 8]
[1, 0, 1, 1, 0, 0]
28


## Convert

In [3]:
def sort_kp(c,w):
    t = [ (c[i],w[i],c[i]/w[i]) for i in range(len(c)) ]
    #t.sort(key=lambda x : x[2])
    c = [p[0] for p in t]
    w = [p[1] for p in t]
    return (c,w)

cw = sort_kp(c,w)

c = np.array(cw[0])
w = np.array(cw[1])

c_dict = { i : c[i] for i in range(0,len(c)) }
w_dict = { i : w[i] for i in range(0,len(w)) }

# Optimization

In [4]:
model = grb.Model("knapsack")
vars = model.addVars(range(0,len(c_dict)),lb=0.0,ub=1.0, \
                     obj=c_dict,vtype=vtype)
model.addConstr(vars.prod(w_dict),grb.GRB.EQUAL,rhs)

fname = 'test.lp'
model.write(fname)

with open(fname) as f:
    content = ''
    for line in f.readlines():
        content = content + line
print(content)



\ Model knapsack
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  0.9063014555369366 C0 + 1.101785411813855 C1 + 0.953227439835186 C2
   + 0.8494630980106754 C3 + 1.070670667517806 C4 + 0.470918422958737 C5
Subject To
 R0: 11 C0 + 7 C1 + 11 C2 + 6 C3 + 12 C4 + 8 C5 = 28
Bounds
Binaries
 C0 C1 C2 C3 C4 C5
End



In [5]:
model.optimize()

Optimize a model with 1 rows, 6 columns and 6 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [6e+00, 1e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+01]
Presolve time: 0.00s
Presolved: 1 rows, 6 columns, 6 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)
Found heuristic solution: objective 2.7089920

Root relaxation: objective 2.157133e+00, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.15713    0    1    2.70899    2.15713  20.4%     -    0s
     0     0 infeasible    0         2.70899    2.70899  0.00%     -    0s

Cutting planes:
  Gomory: 1
  Cover: 2

Explored 1 nodes (1 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 2.70899 

Optimal solution found (toleran

# Solution

In [6]:
x = np.array([v.X for v in vars.values()])

print('objective:','{:.5f}'.format(sum(x*c)))
print('constraint:',sum(x*w),'=',rhs)
print('solution:',x)

try:
    c = c.tolist()
    w = w.tolist()
except:
    pass

for i in range(len(c)):
    print('{:.0f} -> {:4.5f}   {:.0f}'.format(i,c[i],w[i]))

objective: 2.70899
constraint: 28.0 = 28
solution: [ 1. -0.  1.  1. -0. -0.]
0 -> 0.90630   11
1 -> 1.10179   7
2 -> 0.95323   11
3 -> 0.84946   6
4 -> 1.07067   12
5 -> 0.47092   8
